## Sequence Modeling for RUL Prediction

Objective:
Convert per-cycle tabular data into fixed-length temporal windows
to capture degradation progression dynamics.

Evaluation:
Engine-level GroupKFold cross-validation.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/features/train_features_v3.csv")

df.shape

(20631, 48)

In [2]:
WINDOW = 30

In [3]:
from sklearn.preprocessing import StandardScaler

feature_cols = [
    col for col in df.columns
    if col not in ['unit', 'cycle', 'RUL', 'RUL_capped']
]

scaler = StandardScaler()

df[feature_cols] = scaler.fit_transform(df[feature_cols])

In [4]:
from sklearn.preprocessing import StandardScaler

target_scaler = StandardScaler()
df['RUL_scaled'] = target_scaler.fit_transform(df[['RUL_capped']])

In [5]:
X_sequences = []
y_sequences = []
groups = []

for unit in df['unit'].unique():
    engine_data = df[df['unit'] == unit]
    
    features = engine_data[feature_cols].values
    targets = engine_data['RUL_scaled'].values
    
    for i in range(len(engine_data) - WINDOW):
        X_sequences.append(features[i:i+WINDOW])
        y_sequences.append(targets[i+WINDOW-1])
        groups.append(unit)

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)
groups = np.array(groups)

print("X shape:", X_sequences.shape)
print("y shape:", y_sequences.shape)

X shape: (17631, 30, 44)
y shape: (17631,)


In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [7]:
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [8]:
from sklearn.model_selection import GroupKFold

gkf = GroupKFold(n_splits=5)

In [9]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out.squeeze()

### Picking first fold

In [10]:
from sklearn.metrics import mean_squared_error

folds = list(gkf.split(X_sequences, y_sequences, groups))

train_idx, val_idx = folds[0]

X_train, X_val = X_sequences[train_idx], X_sequences[val_idx]
y_train, y_val = y_sequences[train_idx], y_sequences[val_idx]

train_dataset = SequenceDataset(X_train, y_train)
val_dataset   = SequenceDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=128, shuffle=False)

### Initializing model

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTMModel(input_dim=X_sequences.shape[2]).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Short training loop

In [14]:
EPOCHS = 15

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    # ---------------- VALIDATION ----------------
    model.eval()
    val_preds = []
    val_targets = []
    
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            outputs = model(X_batch)
            
            val_preds.extend(outputs.cpu().numpy())
            val_targets.extend(y_batch.cpu().numpy())
    
    # Convert back to real RUL cycles
    val_preds_real = target_scaler.inverse_transform(
        np.array(val_preds).reshape(-1, 1)
    ).flatten()

    val_targets_real = target_scaler.inverse_transform(
        np.array(val_targets).reshape(-1, 1)
    ).flatten()

    rmse = np.sqrt(mean_squared_error(val_targets_real, val_preds_real))

    print(f"Epoch {epoch+1}/{EPOCHS}, RMSE (cycles): {rmse:.4f}")

Epoch 1/15, RMSE (cycles): 23.4697
Epoch 2/15, RMSE (cycles): 23.5710
Epoch 3/15, RMSE (cycles): 23.1249
Epoch 4/15, RMSE (cycles): 23.3875
Epoch 5/15, RMSE (cycles): 23.8454
Epoch 6/15, RMSE (cycles): 23.4593
Epoch 7/15, RMSE (cycles): 23.2713
Epoch 8/15, RMSE (cycles): 23.6544
Epoch 9/15, RMSE (cycles): 23.5752
Epoch 10/15, RMSE (cycles): 23.4086
Epoch 11/15, RMSE (cycles): 23.3801
Epoch 12/15, RMSE (cycles): 23.6895
Epoch 13/15, RMSE (cycles): 23.8166
Epoch 14/15, RMSE (cycles): 23.4594
Epoch 15/15, RMSE (cycles): 23.7063
